# Uni-Tat Iceman Call Center Analytics with Snowflake Cortex AI

## Setting up session

In [ ]:
import pandas as pd
import json
import numpy as np
import time
from datetime import datetime, timedelta
import streamlit as st

from snowflake.snowpark import Session, DataFrame
from snowflake.snowpark import functions as F
from snowflake.snowpark import types as T
from snowflake.snowpark.version import VERSION

session = get_active_session()
session.use_role("iceman_call_center_analytics_role")
session.use_schema("analytics")

session.query_tag = {"origin":"sf_sit", "name":"call_center_analytics_2", "version":{"major":1, "minor":0}, "attributes":{"is_quickstart":1, "source":"notebook"}}

## List the audio files in the stage

In [ ]:
LIST @audio_files/;

## Preview audio files with Streamlit

In [ ]:
stage_path = "@iceman_call_center_analytics_db.analytics.audio_files/2025-08-15_10-13-35_201-UniTatIce_60110344_68684213_68684213_hpbx1.myrepublic.com.sg-1755224015.2859473.WAV"

with session.file.get_stream(stage_path) as f:
    audio_bytes = f.read()

    st.audio(audio_bytes, format="audio/mpeg", start_time=0)

## Creating table with FILE objects for batch transcription

In [ ]:
CREATE OR REPLACE TABLE audio_files_for_transcription AS
SELECT 
    RELATIVE_PATH as file_path,
    TO_FILE('@audio_files', RELATIVE_PATH) as audio_file,
    SIZE as file_size_bytes,
    LAST_MODIFIED as upload_time,
    REGEXP_SUBSTR(RELATIVE_PATH, '^[^_]+_[^_]+') AS call_id,
    SPLIT_PART(RELATIVE_PATH, '_', 3) AS handler_id
FROM DIRECTORY('@audio_files')
WHERE RELATIVE_PATH ILIKE '%.wav';

In [ ]:
SELECT * FROM audio_files_for_transcription;

## Transcribing audio files

In [ ]:
CREATE OR REPLACE TABLE ai_transcribed_calls AS
SELECT 
    call_id,
    handler_id,
    AI_TRANSCRIBE(audio_file):text::STRING as transcript_text,
    CURRENT_TIMESTAMP() as transcription_timestamp,
    LENGTH(AI_TRANSCRIBE(audio_file):text::STRING) as transcript_length,
    ARRAY_SIZE(SPLIT(AI_TRANSCRIBE(audio_file):text::STRING, ' ')) as word_count,
    CASE 
        WHEN AI_TRANSCRIBE(audio_file):text IS NULL THEN 'FAILED'
        WHEN LENGTH(AI_TRANSCRIBE(audio_file):text::STRING) < 10 THEN 'SHORT'
        ELSE 'SUCCESS'
    END as transcription_status,
    file_path
FROM audio_files_for_transcription
ORDER BY file_size_bytes ASC;  -- Start with smaller files

In [ ]:
SELECT * FROM ai_transcribed_calls;

## Using Snowflake Cortex AI to analyse transcripts

In [ ]:
-- Create table to store transcript analysis
CREATE OR REPLACE TABLE comprehensive_call_analysis AS
SELECT 
    call_id,
    handler_id,
    transcript_text,
    
    -- Sentiment Analysis
    SNOWFLAKE.CORTEX.SENTIMENT(transcript_text) as sentiment_score,
    CASE 
        WHEN SNOWFLAKE.CORTEX.SENTIMENT(transcript_text) > 0.1 THEN 'POSITIVE'
        WHEN SNOWFLAKE.CORTEX.SENTIMENT(transcript_text) < -0.1 THEN 'NEGATIVE'
        ELSE 'NEUTRAL'
    END as sentiment_category,
    
    -- Call Summary
    SNOWFLAKE.CORTEX.SUMMARIZE(transcript_text) as call_summary,
    
    -- Extracting information from call with AI_COMPLETE
    AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt => 'Analyze this call center conversation and extract structured information. Call transcript: ' || transcript_text,
        model_parameters => {'temperature': 0.1, 'max_tokens': 2048},
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'primary_intent': {'type': 'string', 'enum': ['product_enquiry', 'sales', 'delivery_status', 'complaint', 'cancellation', 'other']},
                    'urgency_level': {'type': 'string', 'enum': ['low', 'medium', 'high', 'critical']},
                    'issue_resolved': {'type': 'string', 'enum': ['yes', 'no', 'partial']},
                    'escalation_required': {'type': 'string', 'enum': ['yes', 'no']},
                    'customer_satisfaction': {'type': 'string', 'enum': ['satisfied', 'neutral', 'dissatisfied']},
                    'call_duration_estimate': {'type': 'string', 'enum': ['short', 'medium', 'long']},
                    'key_issues': {'type': 'array', 'items': {'type': 'string'}},
                    'action_items': {'type': 'array', 'items': {'type': 'string'}},
                    'invoice_number': {'type': 'array', 'items': {'type': 'string'}},
                    'monetary_amounts': {'type': 'array', 'items': {'type': 'string'}},
                    'callback_requested': {'type': 'string', 'enum': ['yes', 'no']}
                },
                'required': ['primary_intent', 'urgency_level', 'issue_resolved', 'escalation_required', 'customer_satisfaction']
            }
        }
    ) as call_analysis,
    
    -- Quality score
    TRY_CAST(
        AI_COMPLETE(
            model => 'claude-4-sonnet',
            prompt => 'Rate this call center conversation on a scale of 1-10 for agent performance considering: professionalism, problem-solving, communication clarity, and customer service. Provide only the numeric score (no text). If you cannot determine a score, return null and nothing else: ' || transcript_text,
            model_parameters => {'temperature': 0, 'max_tokens': 10}
       )::VARCHAR AS NUMBER(3,1)
    ) as agent_performance_score,
    
    -- Areas of improvement
    AI_COMPLETE(
        model => 'claude-4-sonnet',
        prompt => 'List 3 specific improvement opportunities for this call center conversation in bullet points: ' || transcript_text,
        model_parameters => {'temperature': 0.3, 'max_tokens': 500}
    ) as improvement_areas,

    CURRENT_TIMESTAMP() as analysis_timestamp
    
FROM ai_transcribed_calls
WHERE transcription_status = 'SUCCESS'
AND transcript_text IS NOT NULL
AND LENGTH(transcript_text) > 50;  -- Filter out very short transcripts (eg. number unavailable, sent to voicemail)

In [ ]:
SELECT * FROM comprehensive_call_analysis;

## Extracting JSON fields from AI prompt

In [ ]:
ALTER TABLE comprehensive_call_analysis 
ADD COLUMN 
    primary_intent STRING,
    urgency_level STRING,
    issue_resolved STRING,
    escalation_required STRING,
    customer_satisfaction STRING;


UPDATE comprehensive_call_analysis
SET
    primary_intent = call_analysis:primary_intent::STRING,
    urgency_level = call_analysis:urgency_level::STRING,
    issue_resolved = call_analysis:issue_resolved::STRING,
    escalation_required = call_analysis:escalation_required::STRING,
    customer_satisfaction = call_analysis:customer_satisfaction::STRING;

In [ ]:
SELECT * FROM comprehensive_call_analysis;

## EDA

### Summary

In [ ]:
SELECT 
    COUNT(*) as total_calls,
    ROUND(AVG(sentiment_score), 3) as avg_sentiment,
    ROUND(AVG(agent_performance_score), 1) as avg_agent_score,
    COUNT(DISTINCT handler_id) as unique_agents,
    COUNT(DISTINCT primary_intent) as unique_call_types
FROM comprehensive_call_analysis;

### Agent performance

In [ ]:
SELECT 
    handler_id,
    COUNT(*) as total_calls,
    ROUND(AVG(sentiment_score), 3) as avg_sentiment,
    ROUND(AVG(agent_performance_score), 1) as avg_performance_score,
    
    -- Resolution effectiveness
    SUM(CASE WHEN issue_resolved = 'yes' THEN 1 ELSE 0 END) as resolved_calls,
    ROUND(SUM(CASE WHEN issue_resolved = 'yes' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as resolution_rate,
    
    -- Customer satisfaction
    SUM(CASE WHEN customer_satisfaction = 'satisfied' THEN 1 ELSE 0 END) as satisfied_customers,
    ROUND(SUM(CASE WHEN customer_satisfaction = 'satisfied' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as satisfaction_rate,
    
    -- Escalation patterns
    SUM(CASE WHEN escalation_required = 'yes' THEN 1 ELSE 0 END) as escalations,
    ROUND(SUM(CASE WHEN escalation_required = 'yes' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as escalation_rate
    
FROM comprehensive_call_analysis
GROUP BY handler_id
ORDER BY avg_performance_score DESC;

### Call patterns

In [ ]:
WITH call_patterns AS (
    SELECT 
        primary_intent,
        urgency_level,
        COUNT(*) as call_count,
        ROUND(AVG(sentiment_score), 3) as avg_sentiment,
        ROUND(AVG(agent_performance_score), 1) as avg_agent_score,
        
        -- Resolution patterns
        ROUND(SUM(CASE WHEN issue_resolved = 'yes' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as resolution_rate,
        
        -- Satisfaction patterns
        ROUND(SUM(CASE WHEN customer_satisfaction = 'satisfied' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as satisfaction_rate,
        
        -- Escalation patterns
        ROUND(SUM(CASE WHEN escalation_required = 'yes' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as escalation_rate
        
    FROM comprehensive_call_analysis
    WHERE primary_intent IS NOT NULL AND primary_intent != 'Not Available'
    GROUP BY primary_intent, urgency_level
)
SELECT 
    primary_intent,
    urgency_level,
    call_count,
    avg_sentiment,
    avg_agent_score,
    resolution_rate,
    satisfaction_rate,
    escalation_rate,
    
    -- Flags
    CASE 
        WHEN resolution_rate < 70 THEN '⚠️ Low Resolution'
        WHEN satisfaction_rate < 60 THEN '⚠️ Low Satisfaction'
        WHEN escalation_rate > 30 THEN '⚠️ High Escalation'
        ELSE '✅ Good Performance'
    END as flag
    
FROM call_patterns
ORDER BY call_count DESC;